In [1]:
import numpy as np
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit
from qiskit.visualization import circuit_drawer
from qiskit.circuit import Parameter

# Fix font warnings
plt.rcParams['font.family'] = ['DejaVu Sans', 'Liberation Sans', 'Arial', 'sans-serif']
plt.rcParams['mathtext.default'] = 'regular'

def create_xy_interaction(qc, qubit1, qubit2, beta_param):
    """Add a single XY interaction between two qubits."""
    qc.ry(np.pi/2, qubit1)      # Rotate qubit1 to X-Y plane
    qc.ry(np.pi/2, qubit2)      # Rotate qubit2 to X-Y plane
    qc.cx(qubit1, qubit2)       # First CNOT
    qc.rz(2*beta_param, qubit2) # Z-rotation with 2β parameter
    qc.cx(qubit1, qubit2)       # Second CNOT
    qc.ry(-np.pi/2, qubit1)     # Rotate qubit1 back
    qc.ry(-np.pi/2, qubit2)     # Rotate qubit2 back

def create_3qubit_even_odd_xy_mixer(beta=np.pi/8):
    """Create the 3-qubit even/odd XY mixer with closed ring topology."""
    
    qc = QuantumCircuit(3, name='3Q_EvenOdd_XY_Mixer')
    
    # Create symbolic parameter for display
    beta_param = Parameter('β')
    
    # EVEN LAYER: Connection starting from even index
    qc.barrier(label='Even Layer')
    
    # (0,1) - Only even connection for 3 qubits
    create_xy_interaction(qc, 0, 1, beta_param)
    
    qc.barrier()
    
    # ODD LAYER: Connections starting from odd indices + ring closure
    qc.barrier(label='Odd Layer')
    
    # (1,2) - Odd connection
    create_xy_interaction(qc, 1, 2, beta_param)
    
    # (2,0) - CLOSES THE RING! (can be considered part of odd layer)
    create_xy_interaction(qc, 2, 0, beta_param)
    
    # For simulation, we need to bind the parameter to actual values
    try:
        # Try assign_parameters method
        qc_bound = qc.assign_parameters({beta_param: beta})
    except AttributeError:
        # Fallback: create a separate numerical circuit
        qc_bound = QuantumCircuit(3, name='3Q_EvenOdd_XY_Mixer_Bound')
        
        # Even layer
        qc_bound.barrier()
        create_xy_interaction(qc_bound, 0, 1, 2*beta)  # Use numerical value
        
        qc_bound.barrier()
        
        # Odd layer
        qc_bound.barrier()
        create_xy_interaction(qc_bound, 1, 2, 2*beta)
        create_xy_interaction(qc_bound, 2, 0, 2*beta)  # Closes the ring
    
    return qc, qc_bound

def plot_3qubit_even_odd_xy_mixer():
    """Plot the 3-qubit even/odd XY mixer circuit."""
    
    qc_symbolic, qc_bound = create_3qubit_even_odd_xy_mixer()
    
    # Use symbolic circuit for display (shows 2β)
    try:
        # Method 1: Direct matplotlib circuit drawing
        fig, ax = plt.subplots(figsize=(14, 6))
        qc_symbolic.draw(output='mpl', ax=ax, style='default', fold=None, scale=1.0)
        
        plt.tight_layout()
        plt.savefig('3qubit_even_odd_xy_mixer.png', dpi=600, bbox_inches='tight', 
                    facecolor='white', edgecolor='none')
        
        print("✓ Saved circuit as '3qubit_even_odd_xy_mixer.png' (Method 1)")
        plt.show()
        
    except Exception as e:
        print(f"Method 1 failed: {e}")
        
        # Method 2: Use circuit_drawer but handle differently
        try:
            plt.figure(figsize=(14, 6))
            fig = circuit_drawer(qc_symbolic, output='mpl', style='default', fold=None, scale=1.0)
            
            if fig is not None:
                plt.savefig('3qubit_even_odd_xy_mixer.png', dpi=600, bbox_inches='tight', 
                            facecolor='white', edgecolor='none')
                print("✓ Saved circuit as '3qubit_even_odd_xy_mixer.png' (Method 2)")
                plt.show()
            else:
                print("Circuit drawer returned None")
                
        except Exception as e2:
            print(f"Method 2 also failed: {e2}")
            print("Falling back to text-only representation")
    
    # Always show text representation
    print("\nCircuit text representation:")
    print(qc_symbolic.draw(output='text'))
    
    return qc_symbolic, qc_bound

def demonstrate_3qubit_even_odd_effect():
    """Demonstrate what the 3-qubit even/odd XY mixer does."""
    
    try:
        # Try newer Qiskit first
        from qiskit_aer import Aer
        from qiskit import transpile
        use_aer = True
    except ImportError:
        try:
            # Try older Qiskit
            from qiskit import Aer
            from qiskit import transpile
            use_aer = True
        except ImportError:
            # Fall back to basic quantum_info
            from qiskit.quantum_info import Statevector
            use_aer = False
    
    print("\n3-Qubit Even/Odd XY Mixer Effect:")
    print("="*40)
    print("Using β = π/6 for demonstration")
    print("Testing layered ring connectivity")
    print()
    
    # Test states that show the ring connectivity
    test_states = ['001', '010', '100']
    
    for initial_state in test_states:
        # Create circuit with initial state
        qc = QuantumCircuit(3)
        
        # Prepare initial state
        for i, bit in enumerate(initial_state):
            if bit == '1':
                qc.x(i)
        
        # Add 3-qubit even/odd XY mixer
        _, xy_mixer_bound = create_3qubit_even_odd_xy_mixer(beta=np.pi/6)
        qc = qc.compose(xy_mixer_bound)
        
        if use_aer:
            # Use Aer backend
            backend = Aer.get_backend('statevector_simulator')
            transpiled_qc = transpile(qc, backend)
            job = backend.run(transpiled_qc)
            result = job.result()
            statevector = result.get_statevector()
        else:
            # Use quantum_info Statevector
            statevector = Statevector.from_instruction(qc)
        
        print(f"\nInitial state |{initial_state}⟩:")
        
        # Display final state probabilities
        probs = np.abs(statevector)**2
        states = ['|000⟩', '|001⟩', '|010⟩', '|011⟩', '|100⟩', '|101⟩', '|110⟩', '|111⟩']
        
        for i, (state, prob) in enumerate(zip(states, probs)):
            if prob > 0.001:
                print(f"  {state}: {prob:.3f}")

def explain_3qubit_even_odd_pattern():
    """Explain the 3-qubit even/odd mixer pattern."""
    
    print("\n3-QUBIT EVEN/ODD MIXER PATTERN:")
    print("="*40)
    
    print("EVEN LAYER:")
    print("  • (0,1): Connect first pair (even index start)")
    print("  → Creates one 2-qubit cluster")
    print()
    
    print("ODD LAYER:")
    print("  • (1,2): Connect to third qubit (odd index start)")
    print("  • (2,0): Close the ring back to start")
    print("  → Links all qubits into complete ring")
    print()
    
    print("RESULTING RING TOPOLOGY:")
    print("  Final connectivity: 0-1-2-0")
    print("  Ring path: 0 → 1 → 2 → 0")
    print("  Each qubit has exactly 2 neighbors")
    print()
    
    print("LAYERED APPROACH BENEFITS:")
    print("  ✓ Systematic structure (even then odd)")
    print("  ✓ Clear presentation (easy to understand)")
    print("  ✓ Parallelizable within layers")
    print("  ✓ Same pattern scales to larger systems")
    print("  ✓ Complete ring connectivity guaranteed")

def visualize_3qubit_connectivity():
    """Create a visual representation of the 3-qubit connectivity."""
    
    print("\n3-QUBIT CONNECTIVITY VISUALIZATION:")
    print("="*45)
    
    print("Qubit Layout:")
    print("      q1")
    print("     ╱  ╲")
    print("   q0 ── q2")
    print()
    
    print("Layer Breakdown:")
    print("  Even Layer: (0,1)")
    print("  Odd Layer:  (1,2), (2,0)")
    print()
    
    print("Complete Ring Path:")
    print("  0 → 1 → 2 → 0")
    print("  ↑___________|")
    print()
    
    print("Comparison with Previous 3-Qubit Versions:")
    print("  • Chain:     (0,1), (1,2)           - no ring")
    print("  • Ring:      (0,1), (1,2), (2,0)    - all at once")
    print("  • All-to-All: (0,1), (0,2), (1,2)   - different pairs")
    print("  • Even/Odd:  (0,1) | (1,2), (2,0)   - layered approach")

def compare_3qubit_approaches():
    """Compare different 3-qubit mixer topologies."""
    
    print("\n3-QUBIT TOPOLOGY COMPARISON:")
    print("="*40)
    
    print("1. CHAIN TOPOLOGY:")
    print("   Interactions: (0,1), (1,2)")
    print("   Pros: Simple, sequential")
    print("   Cons: No ring closure")
    print()
    
    print("2. RING TOPOLOGY:")
    print("   Interactions: (0,1), (1,2), (2,0) - all together")
    print("   Pros: Complete ring, simultaneous")
    print("   Cons: Less structured")
    print()
    
    print("3. ALL-TO-ALL TOPOLOGY:")
    print("   Interactions: (0,1), (0,2), (1,2)")
    print("   Pros: Maximum connectivity")
    print("   Cons: Different connectivity pattern")
    print()
    
    print("4. EVEN/ODD RING (Current):")
    print("   Interactions: (0,1) | (1,2), (2,0) - layered")
    print("   Pros: Ring + systematic layers + presentation-friendly")
    print("   Cons: Slightly deeper circuit")
    print()
    
    print("Best for Presentations: Even/Odd Ring")
    print("  → Clear structure, easy to explain, systematic approach")

def main():
    """Generate and demonstrate the 3-qubit even/odd XY mixer."""
    
    print("3-QUBIT EVEN/ODD XY MIXER WITH CLOSED RING")
    print("="*55)
    print("Creating 3-qubit even/odd XY mixer...")
    print("Perfect size for presentations!")
    print("Even Layer: (0,1)")
    print("Odd Layer:  (1,2), (2,0) ← closes ring")
    print("Ring topology: 0-1-2-0")
    print()
    
    # Plot the circuit
    qc_symbolic, qc_bound = plot_3qubit_even_odd_xy_mixer()
    
    # Demonstrate the effect
    demonstrate_3qubit_even_odd_effect()
    
    # Explain the pattern
    explain_3qubit_even_odd_pattern()
    
    # Show connectivity
    visualize_3qubit_connectivity()
    
    # Compare approaches
    compare_3qubit_approaches()
    
    print("\n" + "="*55)
    print("3-QUBIT EVEN/ODD RING QAOA MIXER CONCEPT:")
    print("✓ Two layers: Even pair + Odd connections")
    print("✓ Perfect for presentations: clear and manageable")
    print("✓ Complete ring: every qubit connected in loop")
    print("✓ Systematic: shows layered approach concept")
    print("✓ Scalable: same pattern works for 6, 9, 12+ qubits")
    print("✓ QAOA ready: single β parameter controls all interactions")

if __name__ == "__main__":
    main()